<a href="https://colab.research.google.com/github/Nitesh-061/Optimize_CNNmodels-Keras-Tuner/blob/main/keras_tuner_fmnistDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=446eb1bf75b4a24713ecc6782ca34c9268d3a172fd2296bf1f1a6e5918f8c3cc
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=ffa29706bd324fda7b8586dab1cfe3e73b57db33db021af4cd2b81463a8490b0
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.3.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4272 - accuracy: 0.8477 - val_loss: 0.3155 - val_accuracy: 0.8840
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2789 - accuracy: 0.8994 - val_loss: 0.2811 - val_accuracy: 0.9022
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2216 - accuracy: 0.9189 - val_loss: 0.2550 - val_accuracy: 0.9092


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3832 - accuracy: 0.8612 - val_loss: 0.3097 - val_accuracy: 0.8900
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2412 - accuracy: 0.9109 - val_loss: 0.2620 - val_accuracy: 0.9068
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1812 - accuracy: 0.9329 - val_loss: 0.2429 - val_accuracy: 0.9157


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4139 - accuracy: 0.8504 - val_loss: 0.3097 - val_accuracy: 0.8822
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2667 - accuracy: 0.9024 - val_loss: 0.2658 - val_accuracy: 0.8992
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2139 - accuracy: 0.9189 - val_loss: 0.2778 - val_accuracy: 0.9052


Epoch 1/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.5165 - accuracy: 0.8099 - val_loss: 0.5107 - val_accuracy: 0.8157
Epoch 2/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.3894 - accuracy: 0.8565 - val_loss: 0.3834 - val_accuracy: 0.8582
Epoch 3/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.3665 - accuracy: 0.8620 - val_loss: 0.4150 - val_accuracy: 0.8445


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4578 - accuracy: 0.8363 - val_loss: 0.3565 - val_accuracy: 0.8668
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3351 - accuracy: 0.8752 - val_loss: 0.3241 - val_accuracy: 0.8833
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2914 - accuracy: 0.8931 - val_loss: 0.3151 - val_accuracy: 0.8853


INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        57648     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,661,354
Trainable params: 2,661,354
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1311 - accuracy: 0.9507 - val_loss: 0.2649 - val_accuracy: 0.9088
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0908 - accuracy: 0.9661 - val_loss: 0.2977 - val_accuracy: 0.9120
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0641 - accuracy: 0.9762 - val_loss: 0.3250 - val_accuracy: 0.9120
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0477 - accuracy: 0.9826 - val_loss: 0.3705 - val_accuracy: 0.9112
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0348 - accuracy: 0.9872 - val_loss: 0.4670 - val_accuracy: 0.9053
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0280 - accuracy: 0.9900 - val_loss: 0.5320 - val_accuracy: 0.9118
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0253 - accuracy: 0.9911 - val_loss: 0.4986 - val_accuracy